In [1]:
#THIS FILE 

#basketall hoop is four feet over the court can now use this to calculate shote distance

In [223]:
from bs4 import BeautifulSoup
import requests
import re
import math
import pandas as pd
import numpy as np
import time

def get_cbb_teams():
    team_names = []
    team_ids = []
    team_conferences = []
    page = requests.get('https://www.espn.com/mens-college-basketball/teams')

    soup = BeautifulSoup(page.content, 'html.parser')

    # print(soup)
    conference_divs = soup.findAll("div", {"class": "mt7"})

    for conference in conference_divs:
        conference_espn_name = conference.find('div').text
        teams = conference.findAll('div', {'class': 'pl3'})
        for team in teams:
            info = team.find('a')
            team_id = re.search('id/(.*)/', info['href']).group(1)
            team_ids.append(team_id)

            team_name = info.text 
            team_names.append(team_name)

            team_conferences.append(conference_espn_name)

    teams_df = pd.DataFrame(np.array([team_ids, team_names, team_conferences]).T, columns = ['ESPN_id', 'Name', 'Conference'])

    return teams_df

teams_df = get_cbb_teams()

for name in teams_df.Name:
    print(name)


Bellarmine Knights
Central Arkansas Bears
Eastern Kentucky Colonels
Florida Gulf Coast Eagles
Jacksonville Dolphins
Jacksonville State Gamecocks
Kennesaw State Owls
Liberty Flames
Lipscomb Bisons
North Alabama Lions
North Florida Ospreys
Stetson Hatters
Albany Great Danes
Binghamton Bearcats
Hartford Hawks
Maine Black Bears
NJIT Highlanders
New Hampshire Wildcats
Stony Brook Seawolves
UMBC Retrievers
UMass Lowell River Hawks
Vermont Catamounts
Cincinnati Bearcats
East Carolina Pirates
Houston Cougars
Memphis Tigers
SMU Mustangs
South Florida Bulls
Temple Owls
Tulane Green Wave
Tulsa Golden Hurricane
UCF Knights
Wichita State Shockers
Davidson Wildcats
Dayton Flyers
Duquesne Dukes
Fordham Rams
George Mason Patriots
George Washington Colonials
La Salle Explorers
Rhode Island Rams
Richmond Spiders
Saint Joseph's Hawks
Saint Louis Billikens
St. Bonaventure Bonnies
UMass Minutemen
VCU Rams
Boston College Eagles
Clemson Tigers
Duke Blue Devils
Florida State Seminoles
Georgia Tech Yellow Jack

In [408]:
def get_team_id(html_with_href, search_type = None, search_class = None):
    if search_class == None:  
        try:
            return re.search('id/(.*)/', html_with_href['href']).group(1)
        except AttributeError:
            print("This game is likely not exclusively between division one teams")
    else:
        try:
            return re.search('id/(.*)/', html_with_href.findAll(search_type, {'class': search_class})[0]['href']).group(1)
        except AttributeError:
            print("This game is likely not exclusively between division one teams")


#get game links to scrape with other function
def get_schedule(team_id, year = 2022, already_scraped = []):
    base_schedule_url = 'https://www.espn.com/mens-college-basketball/team/schedule/_/id/'
    #init result
    games_info = []
    #if team is input as the ESPN_id already
    if type(team_id) != int:
        team_id = teams_df[teams_df['Name'] == team_id]['ESPN_id'].iloc[0]
    
    #make the schedule url to pull link info from
    #year is the year the season ends in so
    #for example the year that the NCAA tournament did not happen (2019-20) would be input as 2020
    full_url = base_schedule_url + str(team_id) + '/season/' + str(year)
    page = requests.get(full_url)
    
    soup = BeautifulSoup(page.content, 'html.parser')

    game_links = soup.findAll('div', {'class': 'Card__Content'})[0].findAll('tr')#[0]#.findAll('li')
    schedule = []
        
    postseason = True
    for item in game_links:
        if item.text == 'Regular Season':
            postseason = False
            
        try:
            score_part = item.find('span', {'class': 'ml4'})
            game_id = re.search('gameId/([0-9]*)', score_part.find('a')['href']).group(1)
            if game_id in already_scraped:
                continue
            else:
                already_scraped.append(game_id)
            
            opponent_info = item.find('div').findAll('span')
            if opponent_info[0].text == 'vs':
                home_team = team_id
                try:
                    away_team = get_team_id(opponent_info[2].find('a'))
                except:
                    away_team = None
            else:
                away_team = team_id
                try:
                    home_team = get_team_id(opponent_info[2].find('a'))
                except:
                    away_team = None
            
            if item.find('span', {'class': 'fw-bold'}).text == 'L':
                win = 0
            else:
                win = 1 #('tr')#[0]#.findAll('li')
            winning_score = re.search('([0-9]+)-([0-9]+)', score_part.text).group(1)
            losing_score =  re.search('([0-9]+)-([0-9]+)', score_part.text).group(2)
            date = re.search('[A-z]{3}, [A-z]{3} [0-9]{1,2}', item.text)[0]
            schedule.append([home_team, away_team, game_id, win, date, winning_score, losing_score, postseason])
            
        except AttributeError:
            print("row not for scraping")
            
    
    ret = pd.DataFrame(schedule, columns = ['home_team', 'away_team', 'gameid', 'win', 'date', 'winning_score', 'losing_score', 'postseason'])
    ret['date'] = ret['date'].apply(lambda x: x + f' {year-1}' if 'Oct' in x or 'Nov' in x or 'Dec' in x else x + f' {year}')         
    ret.date = pd.to_datetime(ret.date, format='%a, %b %d %Y')
    
    return ret, already_scraped
        
        
#schedule_df = get_schedule('Georgia State Panthers', 2018)

#print(schedule_df)


def get_season_schedule(teams_df = False, year = 2022):
    if not teams_df:
        teams_df = get_cbb_teams()
    
    season_schedule, already_scraped = get_schedule(teams_df.Name[0], year)
    
    for team_id in teams_df.Name[1:len(teams_df.Name)]:
        time.sleep(2)
        team_schedule, already_scraped = get_schedule(team_id, year, already_scraped)
        season_schedule = season_schedule.append(team_schedule, ignore_index=True)
        
    
    return season_schedule
        

#get_schedule("Duke Blue Devils",2022)
print(season_schedule.shape)
season_schedule = get_season_schedule()
duke_schedule = get_schedule("Duke Blue Devils",2022)
print(season_schedule)
duke_schedule

# get_schedule(91,2022)


(9393, 7)
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row not for scraping
row

(Empty DataFrame
 Columns: [home_team, away_team, gameid, win, date, winning_score, losing_score, postseason]
 Index: [],
 ['401369133',
  '401369145',
  '401369154',
  '401369156',
  '401369163',
  '401365029',
  '401386970',
  '401369177',
  '401385794',
  '401369196',
  '401369195',
  '401369207',
  '401369211',
  '401369216',
  '401369238',
  '401369244',
  '401369253',
  '401369256',
  '401369262',
  '401406670',
  '401369273',
  '401369274',
  '401369280',
  '401369286',
  '401369296',
  '401369303',
  '401369306',
  '401369310',
  '401369134',
  '401369140',
  '401369153',
  '401369160',
  '401369168',
  '401369169',
  '401371700',
  '401369178',
  '401369185',
  '401369212',
  '401369215',
  '401369225',
  '401369229',
  '401369232',
  '401369239',
  '401369250',
  '401369257',
  '401369272',
  '401369275',
  '401369281',
  '401369287',
  '401369294',
  '401369300',
  '401369311',
  '401412530',
  '401385112',
  '401385911',
  '401373026',
  '401372209',
  '401370936',
  '40137

In [394]:
base_playbyplay_url = "http://www.espn.com/mens-college-basketball/playbyplay?gameId="
summary_url = "http://www.espn.com/mens-college-basketball/game?gameId="
base_date_loc = 'https://www.espn.com/mens-college-basketball/game?gameId='

        
#i should probably take much of this code out and make it different functions called by this one

#can take list of games for multiple teams or just one
#can use list(test_schedule['game_id']) on the pervious dataframe to return a list of game ids
#also takes list of games already scraped in case you are doing a large amount of teams and want to prevent game duplication
def get_game_data(games_list, already_scraped = [], box_score = True, play_by_play = True, date_location = True, get_shot_data = True):
    ret_list = []
    play_by_play_data_init = 1
    descriptive_data_init = 1
    game_box_score_init = 1
    shot_data_init = 1
    i = 0
    
    unavailable_shot_charts = 0
    for game_id in games_list:
        print(i)
        i = i+1
        #was this game already scrapped
        if game_id in already_scraped:
            continue
        
        time.sleep(3)
        
        if play_by_play:
            if play_by_play_data_init:
                play_by_play_data = get_play_by_play(game_id)
                play_by_play_data_init = 0
            else:
                play_by_play_data = play_by_play_data.append(get_play_by_play(game_id))

        #these are stored in tables but its not perfect so pd.read_html will not be used
        if box_score:
            if game_box_score_init:
                game_box_score, half_splits = get_box_score(game_id)
                game_box_score_init = 0
            else:
                game_box_score_add, half_splits_add = get_box_score(game_id)
                game_box_score = game_box_score.append(game_box_score_add)
                half_splits = half_splits.append(half_splits_add)

                #have to finish putting this info into a list to become a df
        if date_location:
            if descriptive_data_init:
                descriptive_data = get_descriptive_info(game_id)
                descriptive_data_init = 0
            else:
#                 print('here')
#                 print(descriptive_data)
                descriptive_data = descriptive_data.append(get_descriptive_info(game_id))
#                 print(get_descriptive_info(game_id))
#                 print(game_id)
#                 print(descriptive_data)

        if get_shot_data:
            if shot_data_init:
                try:
                    #print(game_id)
                    shot_data = get_shot_info(game_id)
                    if not isinstance(shot_data, str):
                        shot_data_init = 0
                except AttributeError:
                    unavailable_shot_charts = unavailable_shot_charts + 1
                    print('No Shot Chart Available -', unavailable_shot_charts)
            else:
                #print(game_id)
                try:
                    shot_data = shot_data.append(get_shot_info(game_id, max(shot_data.in_line_index)))
                except AttributeError:
                    unavailable_shot_charts = unavailable_shot_charts + 1
                    print('No Shot Chart Available -', unavailable_shot_charts)
            
        already_scraped.append(game_id)
    if box_score is False:
        play_by_play_data = None
    if play_by_play is False:
        game_box_score = None
    if date_location is False:
        half_splits = None
        descriptive_data = None
    if get_shot_data is False:
        shot_data = None
    ret_list = [play_by_play_data, game_box_score, half_splits, descriptive_data, shot_data]
    return ret_list

#     full_url = base_schedule_url + str(team) + '/season/' + str(year)
#     page = requests.get(full_url)
    
#     soup = BeautifulSoup(page.content, 'html.parser')
#duke_data = get_game_data(duke_schedule.gameid, [], False, False, False)
season_to_date_2022 = get_game_data(season_schedule.gameid, [], False, False, False)


0
No Shot Chart Available - 1
1
No Shot Chart Available - 2
2
No Shot Chart Available - 3
3
4
5
No Shot Chart Available - 4
6
No Shot Chart Available - 5
7
No Shot Chart Available - 6
8
No Shot Chart Available - 7
9
No Shot Chart Available - 8
10
No Shot Chart Available - 9
11
No Shot Chart Available - 10
12
No Shot Chart Available - 11
13
No Shot Chart Available - 12
14
No Shot Chart Available - 13
15
No Shot Chart Available - 14
16
No Shot Chart Available - 15
17
No Shot Chart Available - 16
18
No Shot Chart Available - 17
19
No Shot Chart Available - 18
20
No Shot Chart Available - 19
21
No Shot Chart Available - 20
22
No Shot Chart Available - 21
23
No Shot Chart Available - 22
24
No Shot Chart Available - 23
25
No Shot Chart Available - 24
26
No Shot Chart Available - 25
27
No Shot Chart Available - 26
28
No Shot Chart Available - 27
29
No Shot Chart Available - 28
30
No Shot Chart Available - 29
31
No Shot Chart Available - 30
32
No Shot Chart Available - 31
33
No Shot Chart Avai

In [313]:
def get_box_score(game_id):

    full_box_url = 'https://www.espn.com/mens-college-basketball/boxscore?gameId=' + str(game_id)
    page = requests.get(full_box_url)
    soup = soup = BeautifulSoup(page.content, 'html.parser')
    tables = soup.findAll('tbody')

    away_team = soup.findAll('div', {'class': 'team-name'})[0].text
    home_team = soup.findAll('div', {'class': 'team-name'})[1].text


    #revisit storage of this later and how it should be broken down
    half_breakdowns = tables[0].findAll('tr')
    home_away = 'away'
    half_splits_list = []
    columns = ['team_id', 'team_abbreviation']
    k = 0

    for row in half_breakdowns:
        
        team_id = re.search('id/(.*)/',soup.findAll('a', {'class': 'team-name'})[k]['href']).group(1)

        temp_data = row.findAll('td')
        if home_away == 'away':
            away_team_abrev = temp_data[0].text
            home_away = 'home'
            result = [team_id, away_team_abrev]
        else:
            home_team_abrev = temp_data[0].text
            result = [team_id, home_team_abrev]

        temp_data = temp_data[1:]
        for item in temp_data:
            result.append(item.text)

        half_splits_list.append(result)
        k = k + 1
        
    columns.extend(['half_'+str(i+1) if i < 2 else 'OT_'+str(i-2) for i in range(len(half_splits_list[0])-3)])
    columns.append('total')
    half_splits_df = pd.DataFrame(half_splits_list, columns = columns)
    
    #break down boxscores from players on both teams now
    #broken up into two tables but four tbodys (starters and bench on both teams)
    #skip first table (the previous one broken down)
    #there is also a table for where teams are in their conference but we do not need or want that

    #player data columns in order are player_name, position, starter, minutes, fg_made, fg_attempted, threes_made,
    #threes_attempted, ft_made, ft_attempted, oreb, dreb, ast, stl, blk, to, pf, pts
    #total rebounds is not taken as it can be easily replicated but points is kept as it is so frequently used
    game_box_score = []
    for i in range(1,5):
        working_table = tables[i]
        for tr in working_table.findAll('tr'):
            try: 
                'highlight' in tr.get('class')
                continue
            except:
                pass
            player_name = tr.find('span').text
            player_name2 = re.search('/id/([0-9]+)/(.*)', tr.find('a')['href'])
            player_id = player_name2.group(1)
            #print(player_name2.group(2))#/id/4592187/wendell-moore-jr
            player_position = tr.find('span', {'class': 'position'}).text


            player_level_data = [ player_id, player_position]
            numeric_info = []

            #is the player a starter or a bench player?
            if i%2 == 0:
                player_level_data.append(False)
            else:
                player_level_data.append(True)

            if i < 3:
                player_level_data.append(half_splits_list[0][1])
            else:
                player_level_data.append(half_splits_list[1][1])

            #get number of made and attempts
            for td in tr.findAll('td')[1:]:
                if (('3pt' in td.get('class')) or ('ft' in td.get('class')) or ('fg' in td.get('class'))):
                    player_level_data.append(re.search('(\d*)-(\d*)',td.text).group(1))
                    player_level_data.append(re.search('(\d*)-(\d*)',td.text).group(2))

                elif ('reb' in td.get('class')):
                    continue
                else:
                    player_level_data.append(td.text)

            game_box_score.append(player_level_data)

    game_box_score = pd.DataFrame(game_box_score, columns = ['player_id', 'position', 'starter', 'team', 'mins', 'fg_made', 'fg_attempted', 'threes_made', 'threes_attempted', 'ft_made', 'ft_attempted', 'oreb', 'dreb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts'])
    return game_box_score, half_splits_df

box_score, half_splits = get_box_score(401253660)


In [18]:
# def get_play_by_play(game_id):
#     test = "http://www.espn.com/mens-college-basketball/playbyplay?gameId=" + str(game_id)
#     test = pd.read_html(test)
#     halves_plus_ots = test[0].shape[1]-2
#     test_times = []
#     away_score = []
#     home_score = []
#     plays = []

#     #home team score on the right fyi
#     for i in range(1, halves_plus_ots+1):
#         test[i].drop(['team','Unnamed: 4'], axis = 1, inplace = True)
#         for j in range(len(test[i]['time'])):
#             time_remaining_info = re.search('(\d*):(\d*)', test[i].loc[j]['time'])
#             if i == 1:
#                 time_remaining = int(time_remaining_info.group(1))*60 + int(time_remaining_info.group(2)) + 1200
#             else:
#                 time_remaining = int(time_remaining_info.group(1))*60 + int(time_remaining_info.group(2))
#             test_times.append(time_remaining)

#             score_info = re.search('(\d*) - (\d*)', test[i].loc[j]['SCORE'])
#             away_score.append(score_info.group(1))
#             home_score.append(score_info.group(2))
#             plays.append(test[i].loc[j]['PLAY'])


#     play_by_play_game_df = pd.DataFrame({'time_remaining': test_times,
#                                         'away_score': away_score,
#                                         'home_score': home_score,
#                                         'play': plays})

#     return play_by_play_game_df
    
# get_play_by_play(400840367)

,time_remaining,away_score,home_score,play
0,2385,0,0,Kieston Standfield Turnover.
1,2362,0,0,Foul on Kieston Standfield.
2,2362,0,0,Markus Crider missed Free Throw.
3,2362,0,0,Georgia St Deadball Team Rebound.
4,2362,0,0,Markus Crider missed Free Throw.
...,...,...,...,...
343,41,52,73,Primetime Hamilton Defensive Rebound.
344,34,52,73,Condell Woodson missed Three Point Jumper.
345,34,52,73,Tevion Pope Offensive Rebound.
346,29,54,73,Tevion Pope made Jumper.


In [253]:
test = 'https://www.espn.com/mens-college-basketball/game?gameId=400840367'
page = requests.get(full_dateloc_url)
soup = BeautifulSoup(page.content, 'html.parser')
soup.findAll('article', {'class', 'game-flow'})
# pd.read_html(test)

[<article class="sub-module game-flow"><header class="bordered"><h1>Game Flow</h1></header><div class="content"><div class="comp game-flow" id="score-flow-container"><div id="gameFlow-view" style="display:block;position:relative;"><div data-away-id="109014" data-awayabbrev="MGAS" data-awaycolor="a8a8a8" data-behavior="game_flow" data-home-id="2247" data-homeabbrev="GAST" data-homecolor="1e539a" data-mininperiod="20" data-numperiods="2" data-plays='[{"period":1,"seconds":0,"homeScore":0,"awayScore":0,"text":"","clock":"12:00"},{"period":1,"seconds":86,"homeScore":0,"awayScore":2,"teamId":"109014","id":"400840367101816501","text":"Kieston Standfield made Jumper. Assisted by Jacob Gibson.","clock":"18:34"},{"period":1,"seconds":129,"homeScore":1,"awayScore":2,"teamId":"2247","id":"400840367101824802","text":"Jeremy Hollowell made Free Throw.","clock":"17:51"},{"period":1,"seconds":129,"homeScore":2,"awayScore":2,"teamId":"2247","id":"400840367101824803","text":"Jeremy Hollowell made Free 

In [170]:
from lxml import html
from lxml.cssselect import CSSSelector

import json

css = CSSSelector

win_pct_marker = "espn.gamepackage.probability.data = "

def is_wins(x):
    return win_pct_marker in x

def extract_other_wins(doc):
    scripts = css("script")(doc)
    target_script = next(script for script in scripts if is_wins(script.text_content()))
    js = target_script.text_content()
    target_line = next(line for line in js.splitlines() if is_wins(line))
    start = target_line.find(win_pct_marker)
    data = target_line[start + len(win_pct_marker):-1]
    parsed = json.loads(data)
    return parsed

def get_play_by_play(game_id):
    src = requests.get('https://www.espn.com/mens-college-basketball/game?gameId='+str(game_id)).content
    doc = html.fromstring(src)
    other_win_pcts = extract_other_wins(doc)
    other_win_pcts
    results = []
    # columns in order are play, play type, home score, away score, secs remaining, home win percentage, period (half or ot)
    for event in other_win_pcts:
        time = re.search('(\d*):(\d*)', event['play']['clock']['displayValue'])

        secs_remain = int(time.group(1))*60 + int(time.group(2))
        if event['play']['period']['number'] == 1:
            secs_remain = secs_remain + 20*60

        temp = [event['play']['text'],
                event['play']['type']['text'],
                event['play']['homeScore'], 
                event['play']['awayScore'],
                secs_remain,
                event['homeWinPercentage'],
                event['play']['period']['number']]

        results.append(temp)

    #Pierre Crockrell II

    return pd.DataFrame(results, columns = ['play',
                                            'play_event',
                                            'home_score',
                                            'away_score',
                                            'seconds_remaining',
                                            'home_win_percentage',
                                           'period_number'])

get_play_by_play(401284857)
# other_win_pcts
# {'play': {'period': {'displayValue': '1st Half', 'number': 1},
#    'homeScore': 0,
#    'awayScore': 0,
#    'text': 'Jump Ball won by Pacific',
#    'clock': {'displayValue': '20:00'},
#    'type': {'id': '615', 'text': 'Jumpball'}},
#   'homeWinPercentage': 0.028,
#   'playId': '401284857101799901',
#   'tiePercentage': 0}

,play,play_event,home_score,away_score,seconds_remaining,home_win_percentage,period_number
0,Jump Ball won by Pacific,Jumpball,0,0,2400,0.028,1
1,Jordan Bell missed Three Point Jumper.,JumpShot,0,0,2385,0.027,1
2,Jalen Suggs Defensive Rebound.,Defensive Rebound,0,0,2385,0.026,1
3,Jalen Suggs missed Three Point Jumper.,JumpShot,0,0,2374,0.029,1
4,Broc Finstuen Defensive Rebound.,Defensive Rebound,0,0,2374,0.029,1
...,...,...,...,...,...,...,...
326,Pacific Deadball Team Rebound.,Dead Ball Rebound,58,74,29,0.001,2
327,Pierre Crockrell II missed Free Throw.,MadeFreeThrow,58,74,29,0.001,2
328,Ben Gregg Defensive Rebound.,Defensive Rebound,58,74,29,0.001,2
329,Julian Strawther made Jumper.,JumpShot,58,76,6,0.001,2


In [286]:


def get_descriptive_info(game_id):
    full_dateloc_url = 'https://www.espn.com/mens-college-basketball/game?gameId=' + str(game_id)
    page = requests.get(full_dateloc_url)
    soup = BeautifulSoup(page.content, 'html.parser')
#print(soup)
    loc_details = soup.find('div', {'class', 'location-details'})

    city_state = re.search('([a-zA-Z ]*), ([a-zA-Z]*)', loc_details.find('div', {'class': 'game-location'}).text)
    city = city_state.group(1)
    state = city_state.group(2)
    # print(city, state)
    loc_details = soup.find('article', {'class', 'game-information'})
    # print(loc_details)
    try:
        arena = loc_details.find('figcaption').text.strip()#, {'class', 'game-location'}).text
    except:
        arena = loc_details.find('div', {'class', 'game-location'}).text.strip()

    #print(loc_details)
    game_time = loc_details.find('div', {'class', 'game-date-time'}).find('span')['data-date']
    game_network = loc_details.find('div', {'class', 'game-network'}).text

    network = re.search('Coverage: ([A-z ]*)', game_network).group(1)
    game_attendance = loc_details.find('div', {'class', 'capacity'}).text

    #game_refs = loc_details.find('div', {'class', 'game-info-note__container'}).text

    betting_lines = loc_details.find('div', {'class', 'odds-lines-plus-logo'}).text
    over_under = re.search('Over/Under: ([0-9]{3}.[0,5])', betting_lines).group(1)

    spread_info = re.search('Line: ([A-Z]*) (-[0-9]{1,2}.[0,5])', betting_lines)
    point_spread = spread_info.group(2)
    favorite = spread_info.group(1)
    
    refs = soup.find('span', {'class', 'game-info-note__content'}).text.split(',')
    refs = [ref.strip() for ref in refs]

    ret = [[game_id, city, state, arena, game_time, network, game_attendance, over_under, favorite, point_spread, refs]]
    
    ret = pd.DataFrame(ret, 
                       columns = ['game_id', 'city',
                      'state', 'arena',
                      'game_time', 'network',
                      'game_attendance', 'over_under',
                      'favorite', 'point_spread',
                      'refs'])
    return ret

get_descriptive_info(401373593)

                    
                    

,game_id,city,state,arena,game_time,network,game_attendance,over_under,favorite,point_spread,refs
0,401373593,Storrs,CT,Gampel Pavilion,2022-02-17T01:30Z,CBSSN,"Attendance: 10,167",136.0,CONN,-6.0,"[Brian O'Connell, John Gaffney, Earl Walton]"


In [448]:
# src = requests.get('https://www.espn.com/mens-college-basketball/game?gameId='+str(401284857)).content
# doc = html.fromstring(src) 

# scripts = css("script")(doc)

# target_script = next(script for script in scripts if 'espn.scoreboard' in x script.text_content())
# js = target_script.text_content()
# target_line = next(line for line in js.splitlines() if is_wins(line))
# start = target_line.find(win_pct_marker)
# data = target_line[start + len(win_pct_marker):-1]
# parsed = json.loads(data)
# #print(scripts)





# from lxml import html
# from lxml.cssselect import CSSSelector

# import json

# css = CSSSelector

# win_pct_marker = "espn.gamepackage.probability.data = "


# def is_wins(x):
#     return win_pct_marker in x

# def extract_other_wins(doc):
#     scripts = css("script")(doc)
#     target_script = next(script for script in scripts if is_wins(script.text_content()))
#     js = target_script.text_content()
#     target_line = next(line for line in js.splitlines() if is_wins(line))
#     start = target_line.find(win_pct_marker)
#     data = target_line[start + len(win_pct_marker):-1]
#     parsed = json.loads(data)
#     return parsed

# def get_play_by_play(game_id):
#     src = requests.get('https://www.espn.com/mens-college-basketball/game?gameId='+str(game_id)).content
#     doc = html.fromstring(src)
#     other_win_pcts = extract_other_wins(doc)
    
# get_play_by_play(401284857)
def get_shot_info(game_id, in_line_index = -1):
    full_dateloc_url = 'https://www.espn.com/mens-college-basketball/game?gameId=' + str(game_id)
    page = requests.get(full_dateloc_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    #print(soup)
    shot_chart = soup.find('div', {'class', 'shot-chart'})
    shot_chart = shot_chart.findAll('li')
    # print(loc_details)
    
    game_shot_data = []
    for shot in shot_chart:
        home_away = shot['data-homeaway']
        made = 0 if shot['class'][0] == "missed" else 1
        positions = shot['style']
        y = int(float(re.search('left:([0-9]{1,2}.[0-9]{1})', positions).group(1))*.94)
        if home_away == "home":
            y = 94-y
        x = int(float(re.search('top:([0-9]{1,2}.[0-9]{1})', positions).group(1))*.50)
        text = shot['data-text']
        if made == 0:
            shot_by = re.search('(.+)missed', text).group(1)
            shot_type = re.search('missed ([\w ]*)\.', text).group(1)
            assisted = 0
            assisted_by = None

        else:
            shot_by = re.search('(.+)made', text).group(1)
            shot_type = re.search('made ([\w ]*)\.', text).group(1)
            assisted_by = re.search('Assisted by ([\w ]*)\.', text)
            #print(assisted_by)

            if assisted_by is None:
                assisted_by = None
                assisted = 0
            else:
                assisted_by = assisted_by.group(1)
                assisted = 1
        shooter_id = shot['data-shooter']
        in_line_index = in_line_index + 1
        
        zone = get_shot_zone(shot_type, y, x)
        
        game_shot_data.append([in_line_index, game_id, shooter_id, assisted_by, assisted, shot_type, zone, shot_by, x, y, made, home_away])

    return pd.DataFrame(game_shot_data, columns = ['in_line_index', 'game_id', 'shooter_id', 'assisted_by', 'assisted', 'shot_type', 'zone', 'shot_by', 'x', 'y', 'made', 'home_away'])

    #add game_id
    #make shot_id
    #make assisted another df to save space

#     regex = re.compile('(.+)a red car')
# regex.search("What i want is a red car").group(1) 


def get_shot_zone(shot_type, y, x):
    if shot_type == 'Three Point Jumper':
        if y < 9:
            if x < 24:
                return 1
            else:
                return 2
        else:
            if x < 18:
                return 3
            elif x < 30:
                return 4
            else:
                return 5
    else:
        if y < 10:
            if x < 18:
                return 6
            elif x < 30:
                return 7
            else: return 8
        else:
            if x < 24:
                return 9
            else:
                return 10
        
    
test = get_shot_info(401368416, in_line_index = 50)
max_x = 0
for i, line in test.iterrows():
    #print(line)
    if line['x'] > max_x:
        max_x = line['x']
max_x
    # shots home-team

    
#     for row in df.rows:
#    print row['c1'], row['c2']

# 1 = 'left corner'
# 2 = 'right corner'
# 3 = 'left wing'
# 4 = 'central three'
# 5 = 'right wing'
# 6 = 'left baseline'
# 7 = 'paint'
# 8 = 'right baselline'
# 9 = 'left elbow'
# 10 = 'right elbow'

47.0

In [445]:
def get_player_name(players):
    player_year_list = []
    ret_list = []
    
    for player_id in players:
        full_dateloc_url = 'https://www.espn.com/mens-college-basketball/player/stats/_/id/' + str(player_id)
        time.sleep(.25)
        print(player_id)
        page = requests.get(full_dateloc_url)
        soup = BeautifulSoup(page.content, 'html.parser')

        player_info = soup.find('div', {'class', 'PlayerHeader__Main_Aside'})
        player_info = player_info.findAll('span')
        first_name = player_info[0].text
        last_name = player_info[1].text
    
    
        position_number = soup.find('ul', {'class', 'PlayerHeader__Team_Info'})
        position_number = position_number.findAll('li')
        if len(position_number) > 2:
            number = position_number[1].text[1:]
            position = position_number[2].text
        else:
            number = None
            position = position_number[1].text
            
        player_bio = soup.find('ul', {'class', 'PlayerHeader__Bio_List'})
        player_bio = player_bio.findAll('li')
        if len(player_bio) > 2:
            height_weight = player_bio[1]
            feet = int(height_weight.text[5]) *12
            inches = int(height_weight.text[8:10].replace('"', ''))
            height = feet+inches
            weight = int(height_weight.text[12:16])

            home = player_bio[2].findAll('div')[2].text
            hometown = home[:home.find(',')]
            homestate = home[home.find(',')+2:]
        else:
            height = None
            weight = None

            #a couple people without year on profile so do len -1 to get hometown
            home = player_bio[len(player_bio)-1].findAll('div')[1].text
            hometown = home[:home.find(',')]
            homestate = home[home.find(',')+2:]
#         else:
#             height = None
#             weight = None

#             home = player_bio[0].findAll('div')[1].text
#             hometown = home[:home.find(',')]
#             homestate = home[home.find(',')+2:]

        ret = [player_id, first_name, last_name, number, position, height, weight, hometown, homestate]

        temp = soup.find('div', {'class', 'ResponsiveTable--fixed-left'})
        temp = temp.find('tbody', {'class', 'Table__TBODY'}).findAll('tr')
        #temp = temp.find('tbody', {'class', 'Table__TBODY'}).findAll('tr')
        for row in temp[:len(temp)-1]:
            try:
                #team = row.find('img')['alt']
                year = row.find('td').text[0:2] + row.find('td').text[5:]
                team_id_info = row.find('a')
                team_id = get_team_id(team_id_info)
                
                player_year_list.append([player_id, team_id, year])
                
            except TypeError:
                print('None D1 School')
            
        
        ret_list.append(ret)
    
    player_year_df = pd.DataFrame(player_year_list, columns = ['player_id',
                                                            'team_id',
                                                            'year'])
    ret_df = pd.DataFrame(ret_list, columns = ['player_id',
                                           'first_name',
                                           'last_name',
                                           'number',
                                           'position',
                                           'height_in',
                                           'weight',
                                           'home_city',
                                           'home_state'])

    return player_year_df, ret_df

player_year_df, player_df = get_player_name(season_to_date_2022[4].shooter_id.unique())
#season_to_date_2022[4].shooter_id.unique()
#     PlayerHeader__Name

#print(season_schedule.date[0:5])
#pd.to_datetime(season_schedule.date + ' 2022', format='%a, %b %d %Y')[0:5]
#season_schedule['date'] = 
#season_schedule['date'].apply(lambda x: x + f' {2015-1}' if 'Oct' in x or 'Nov' in x or 'Dec' in x else x + f' {2015}')


4897013
4700826
4700822
4700829
4067013
4432966
4700823
4700825
4700827
4700831
4395712
4431695
4433255
4432181
4397242
4431705
4683737
4433281
4711313
4565699
4431887
4432817
4397128
4397127
4432848
4432247
4277987
4397131
4576087
4432876
4397129
4278007
4067001
4397469
4593474
4703234
4593473
4431810
4896993
4896991
4397689
4067218
4593106
4432815
4431700
4898374
4683857
4431808
4431830
4683758
4395988
4433617
4397858
4067358
4592249
4700870
4277915
4396688
4280115
4592248
4741885
4700869
4897566
4278483
4278351
4897562
4897563
4702251
4280064
4396697
4592767
4066981
4896463
4896464
4702889
4432191
4593259
4594269
4067063
4278713
4896465
4703244
4593671
4703243
4433059
4067814
4277912
4278993
4703247
4565203
4432846
4397132
4431707
4433561
4702177
4576085
4280015
4706557
4592262
4397715
4700839
4700841
4433177
4397917
4433069
4433599
4433629
4277846
4576060
4897413
4433268
4433606
4701555
4278099
4278884
4592271
4396745
4396747
4700844
4897570
4592272
4397112
4433618
4592427
4590354


In [446]:
# #df.groupby(['domain', 'ID']).count()
# season_schedule.groupby('team').count()


# season_schedule.tail(1)




# # https://www.espn.com/mens-college-basketball/team/roster/_/id/150
# # def get_team_roster







# #len(duke_data)
# duke_data[4].to_csv('duke_shots_21022022.csv')

# duke_data[4]#.in_line_index


# season_to_date_2022[4].groupby('game_id').count()


# player_df.to_csv('player_df.csv')
# season_schedule.to_csv('season_schedule_2022.csv')
# season_to_date_2022[4].to_csv('shot_chart_22022022.csv')
# 1 = 'left corner'
# 2 = 'right corner'
# 3 = 'left wing'
# 4 = 'central three'
# 5 = 'right wing'
# 6 = 'left baseline'
# 7 = 'paint'
# 8 = 'right baselline'
# 9 = 'left elbow'
# 10 = 'right elbow'




# zones_df = pd.DataFrame([[1, 'left corner'], [2, 'right corner'], 
#                          [3, 'left wing'], [4, 'central three'], 
#                          [5, 'right wing'], [6, 'left baseline'],
#                         [7, 'paint'], [8, 'right baseline'],
#                         [9, 'left elbow'], [10, 'right elbow']],
#                        columns = ['zone_id', 'zone_string'])
# zones_df.to_csv('zone_df.csv')

# teams_df.to_csv('teams_df.csv')

print(player_year_df.head)
season_dates_df = pd.DataFrame([[2022, min(season_schedule.date), max(season_schedule.date)]],columns = ['season', 'start', 'end'])
season_dates_df.to_csv('season_dates_df.csv')
player_year_df.to_csv('player_year_df.csv')

<bound method NDFrame.head of      player_id team_id  year
0      4897013      91  2022
1      4700826      91  2021
2      4700826      91  2022
3      4700822      91  2021
4      4700822      91  2022
...        ...     ...   ...
5288   4431697    2253  2022
5289   4432196    2253  2021
5290   4432196    2253  2022
5291   4702029    2253  2021
5292   4702029    2253  2022

[5293 rows x 3 columns]>
